### 特征工程
特征工程时整个机器学习中非常重要的一部分，如何对数据进行特征提取对最终结果影响是深远和巨大的，一般都会优先考虑算法与参数，数据特征才是决定整体结果的上限（垃圾入-->垃圾出），算法与参数实际上是再逼近这个上限的过程。

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import requests
import json
import re
import warnings
warnings.filterwarnings('ignore')

In [2]:
names = ['xqmc','sq','zj','pjjg','fwhx','lc','mj','hxjg','jzlx','fwcx','jzjg',
         'zxqk','lhbl','dt','gpsj','jyqs','scjy','fwyt','fwnx','cqss','dyxx']
house_data = pd.read_excel('../data/data.xls',sheet_name='Sheet3',names=names)

In [3]:
# 数据清洗
house_data.replace('\n','',regex=True)
house_data.head(2)

,xqmc,sq,zj,pjjg,fwhx,lc,mj,hxjg,jzlx,fwcx,...,zxqk,lhbl,dt,gpsj,jyqs,scjy,fwyt,fwnx,cqss,dyxx
0,滨江春盛大江名筑,大江东,126.0,8572,3室2厅1厨1卫,低楼层,147.0,平层,板楼,南,...,毛坯,两梯四户,有,\n\n2021/12/08\n,商品房,\n\n2021/03/31\n,\n\n普通住宅\n,\n\n未满两年\n,\n\n非共有\n,\n\n\n 无抵押\n ...
1,滨江春盛大江名筑,大江东,126.0,8572,3室2厅1厨1卫,低楼层,147.0,平层,板楼,南,...,毛坯,两梯四户,有,\n\n2021/12/08\n,商品房,\n\n2021/03/31\n,\n\n普通住宅\n,\n\n未满两年\n,\n\n非共有\n,\n\n\n 无抵押\n ...


In [31]:
# 地址数据转换为经纬度（浮点型）
full_dz_lists =list( '杭州市' + house_data.sq + house_data.xqmc)
test_dz = full_dz_lists[:10]
# 构建请求头
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36 Edg/106.0.1370.47'
}
# 构建文件（用于存储得到的经纬度值）
fp = open('./dz.txt','a',encoding='utf-8')
for dz in test_dz:
    url = 'https://apis.map.qq.com/jsapi?qt=geoc&addr={}&key=UGMBZ-CINWR-DDRW5-W52AK-D3ENK-ZEBRC&output=jsonp&pf=jsapi&ref=jsapi&cb=qq.maps._svcb3.geocoder0'.format(dz)
    response = requests.get(url=url,headers=headers)
    json_strs = response.text
    # 数据清洗（得到每个房产的经纬度值）
    json_dict = json.loads(json_strs.split('(')[1].split(')')[0])
    # 提取经度值
    lon = json_dict['detail']['pointx']
    # 提取维度值
    lat = json_dict['detail']['pointy']
    full_info = lon + ',' + lat + '\n'
    fp.write(full_info)
fp.close()

In [4]:
house_data_test_10 = house_data.iloc[:10,:]
name = ['lon','lat']
lon_lat = pd.read_csv('dz.txt',names=name)
house_data_test_10['jd'] = lon_lat.lon
house_data_test_10['wd'] = lon_lat.lat
house_data_test_10 =house_data_test_10.iloc[:,2:]
house_data_test_10

,zj,pjjg,fwhx,lc,mj,hxjg,jzlx,fwcx,jzjg,zxqk,...,dt,gpsj,jyqs,scjy,fwyt,fwnx,cqss,dyxx,jd,wd
0,126.0,8572,3室2厅1厨1卫,低楼层,147.00,平层,板楼,南,混合结构,毛坯,...,有,\n\n2021/12/08\n,商品房,\n\n2021/03/31\n,\n\n普通住宅\n,\n\n未满两年\n,\n\n非共有\n,\n\n\n 无抵押\n ...,120.495293,30.274252
1,126.0,8572,3室2厅1厨1卫,低楼层,147.00,平层,板楼,南,混合结构,毛坯,...,有,\n\n2021/12/08\n,商品房,\n\n2021/03/31\n,\n\n普通住宅\n,\n\n未满两年\n,\n\n非共有\n,\n\n\n 无抵押\n ...,120.495293,30.274252
2,117.0,8969,3室2厅1厨2卫,中楼层,130.45,平层,板楼,南,钢混结构,毛坯,...,有,\n\n2022/08/24\n,商品房,\n\n2013/03/29\n,\n\n普通住宅\n,\n\n满五年\n,\n\n非共有\n,\n\n\n 有抵押 100万...,120.299362,30.010227
3,80.0,9105,2室2厅1厨2卫,低楼层,87.87,平层,板塔结合,南,钢混结构,毛坯,...,有,\n\n2022/09/15\n,商品房,\n\n2021/06/24\n,\n\n普通住宅\n,\n\n未满两年\n,\n\n非共有\n,\n\n\n 无抵押\n ...,120.246910,29.993143
4,83.0,9230,2室2厅1厨1卫,高楼层,89.93,平层,板楼,南 北,混合结构,精装,...,无,\n\n2022/08/20\n,商品房,\n\n2010/05/04\n,\n\n普通住宅\n,\n\n满五年\n,\n\n共有\n,\n\n\n 无抵押\n ...,119.727051,30.229900
5,83.0,9230,2室2厅1厨1卫,高楼层,89.93,平层,板楼,南 北,混合结构,精装,...,无,\n\n2022/08/20\n,商品房,\n\n2010/05/04\n,\n\n普通住宅\n,\n\n满五年\n,\n\n共有\n,\n\n\n 无抵押\n ...,119.727051,30.229900
6,112.0,9812,3室2厅1厨2卫,高楼层,114.15,平层,板楼,南,钢混结构,简装,...,无,\n\n2022/07/01\n,商品房,\n\n2006/05/11\n,\n\n普通住宅\n,\n\n满五年\n,\n\n非共有\n,\n\n\n 有抵押 50万元...,119.698560,30.219750
7,59.0,10014,1室1厅1厨1卫,高楼层,58.92,平层,板塔结合,南,混合结构,毛坯,...,有,\n\n2022/09/11\n,商品房,\n\n2022/04/02\n,\n\n普通住宅\n,\n\n未满两年\n,\n\n非共有\n,\n\n\n 无抵押\n ...,120.246910,29.993143
8,83.0,10294,2室2厅1厨1卫,高楼层,80.63,平层,板楼,南,钢混结构,毛坯,...,有,\n\n2021/08/05\n,商品房,\n\n2021/09/21\n,\n\n普通住宅\n,\n\n未满两年\n,\n\n非共有\n,\n\n\n 无抵押\n ...,119.802856,30.011856
9,144.0,10443,4室3厅1厨2卫,高楼层,137.90,跃层,板楼,南,钢混结构,精装,...,无,\n\n2021/08/29\n,商品房,\n\n2006/10/20\n,\n\n普通住宅\n,\n\n满五年\n,\n\n共有\n,\n\n\n 无抵押\n ...,119.718400,30.259470


In [5]:
# 房屋户型（属性）
fwhx = np.unique(house_data_test_10.fwhx)
# 映射法
from sklearn.preprocessing import LabelEncoder
fle = LabelEncoder()
fwhx_labels = fle.fit_transform(house_data_test_10.fwhx)
house_data_test_10['fwhx_labels'] = fwhx_labels
house_data_test_10 = house_data_test_10.drop('fwhx',axis=1)

In [6]:
# 楼层
lc = np.unique(house_data_test_10.lc)
# ONE-HOT 独热编码
# 完成独热编码
lc_ohe_labels = pd.get_dummies(house_data_test_10.lc)
house_data_test_10 = pd.concat([house_data_test_10,lc_ohe_labels],axis=1)
house_data_test_10 = house_data_test_10.drop('lc',axis=1)

In [7]:
house_data_test_10.head()

,zj,pjjg,mj,hxjg,jzlx,fwcx,jzjg,zxqk,lhbl,dt,...,fwyt,fwnx,cqss,dyxx,jd,wd,fwhx_labels,中楼层,低楼层,高楼层
0,126.0,8572,147.00,平层,板楼,南,混合结构,毛坯,两梯四户,有,...,\n\n普通住宅\n,\n\n未满两年\n,\n\n非共有\n,\n\n\n 无抵押\n ...,120.495293,30.274252,3,0,1,0
1,126.0,8572,147.00,平层,板楼,南,混合结构,毛坯,两梯四户,有,...,\n\n普通住宅\n,\n\n未满两年\n,\n\n非共有\n,\n\n\n 无抵押\n ...,120.495293,30.274252,3,0,1,0
2,117.0,8969,130.45,平层,板楼,南,钢混结构,毛坯,一梯两户,有,...,\n\n普通住宅\n,\n\n满五年\n,\n\n非共有\n,\n\n\n 有抵押 100万...,120.299362,30.010227,4,1,0,0
3,80.0,9105,87.87,平层,板塔结合,南,钢混结构,毛坯,一梯一户,有,...,\n\n普通住宅\n,\n\n未满两年\n,\n\n非共有\n,\n\n\n 无抵押\n ...,120.246910,29.993143,2,0,1,0
4,83.0,9230,89.93,平层,板楼,南 北,混合结构,精装,一梯两户,无,...,\n\n普通住宅\n,\n\n满五年\n,\n\n共有\n,\n\n\n 无抵押\n ...,119.727051,30.229900,1,0,0,1
